In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()
strategy.connect()
sim = strategy.retrieve("non_financial_sim")
strategy.disconnect()

In [4]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [5]:
final_sim = prices.merge(sim,on=["year","quarter","week","ticker"],how="left").dropna().reset_index()
final_sim = final_sim[["year","quarter","date","ticker","adjClose","weekly_price_regression_prediction","score"]]

In [6]:
final_sim["delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]

In [21]:
start = final_sim["date"].min()
end = final_sim["date"].max()
date = start
seats = 3
trades = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        date = start
        while date <= end:
            req = float(score/100)
            if date.weekday() > 4:
                date = date + timedelta(days=1)
            try:
                todays_sim = final_sim[(final_sim["date"]==date) & (final_sim["delta"] > 0) \
                                      & (final_sim["score"] >= req)].sort_values("delta",ascending=False)
                if todays_sim.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for seat in range(min(seats,todays_sim.index.size)):
                        try:
                            trade = todays_sim.iloc[seat]
                            ticker = trade["ticker"]
                            buy_price = trade["adjClose"]
                            exit_price = buy_price * (1+delta)
#                             max_date = date + timedelta(days=7)
                            exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                (final_sim["date"] > date) & \
                                              (final_sim["date"] <= max_date) & \
                                              (final_sim["adjClose"] >= exit_price)
                                            ]
                            if exits.index.size < 1:
                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                              (final_sim["year"] == date.year)].sort_values("date",ascending=False)
                                sell_trade = exits.iloc[0]
                            else:
                                sell_trade = exits.iloc[0]
                            trade["sell_date"] = sell_trade["date"]
                            trade["sell_price"] = sell_trade["adjClose"]
                            trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                            trade["score"] = score
                            trade["d"] = d
                            trade["seat"] = seat
                            trades.append(trade)
                            date = sell_trade["date"] + timedelta(days=1)
                        except Exception as e:
                            continue
                    date = date + timedelta(days=1)
            except Exception as e:
                date = date + timedelta(days=1)

  0%|                                                    | 0/3 [00:00<?, ?it/s]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)

100%|████████████████████████████████████████████| 3/3 [00:38<00:00, 12.92s/it]


In [22]:
t = pd.DataFrame(trades)

In [23]:
a = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        score_trades = t[(t["score"]==score) & (t["d"]==d)]
        cash = []
        for seat in range(seats):
            initial = float(100 / seats)
            seat_trades = score_trades[score_trades["seat"]==seat]
            for delta in seat_trades["delta"]:
                initial = initial * (1+delta)
                cash.append(initial)
            analysis = {}
            analysis["score"] = score
            analysis["trades"] = seat_trades.index.size
            analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
            seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
            seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
            analysis["hpr"] = seat_trades["hpr"].mean()
            analysis["seat"] = seat
            analysis["d"] = d
            analysis["pv"] = initial
            a.append(analysis)

  0%|                                                    | 0/3 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/2051318412.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/2051318412.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/2051318412.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [24]:
final = pd.DataFrame(a)

In [25]:
final

,score,trades,winrate,hpr,seat,d,pv
0,70,29,0.862069,4.620690,0,0.00,48.485959
1,70,28,1.000000,45.892857,1,0.00,46.464757
2,70,28,0.964286,100.607143,2,0.00,92.508877
3,80,36,0.861111,39.361111,0,0.00,30.505512
4,80,35,0.971429,43.485714,1,0.00,131.521045
5,80,35,0.942857,70.257143,2,0.00,108.938326
6,90,48,1.000000,20.604167,0,0.00,59.896274
7,90,45,0.977778,48.066667,1,0.00,103.746772
8,90,43,0.953488,56.534884,2,0.00,187.023595
9,70,4,0.750000,252.250000,0,0.05,48.097138


In [26]:
hl = final.pivot_table(index=["score","d"],columns="seat",values="pv")

In [27]:
report = final.merge(hl.reset_index(),on=["score","d"]).drop(["pv","seat"],axis=1).groupby(["score","d"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [28]:
report.sort_values("pv",ascending=False).reset_index()

,score,d,trades,winrate,hpr,0,1,2,pv
0,90,0.10,2.333333,1.000000,1025.722222,476.896885,956.793700,159.465537,1593.156122
1,90,0.05,2.333333,0.888889,898.277778,232.195001,313.268312,309.544753,855.008067
2,70,0.10,4.000000,0.833333,457.250000,59.567941,83.126875,686.734967,829.429783
3,70,0.05,3.666667,0.750000,468.000000,48.097138,76.250962,285.712850,410.060950
4,90,0.00,45.333333,0.977089,41.735239,59.896274,103.746772,187.023595,350.666642
5,80,0.05,6.333333,0.626984,306.412698,54.207207,39.841162,180.510599,274.558968
6,80,0.10,6.333333,0.626984,306.412698,54.207207,39.841162,180.510599,274.558968
7,80,0.00,35.333333,0.925132,51.034656,30.505512,131.521045,108.938326,270.964884
8,70,0.00,28.333333,0.942118,50.373563,48.485959,46.464757,92.508877,187.459593


In [29]:
benchmark = t[(t["score"]==90) & (t["d"]==0)]

In [30]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

  0%|                                                    | 0/3 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/744403605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/744403605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
C:\Users\ericj\AppData\Local\Temp/ipykernel_20656/744403605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [31]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [32]:
sp5.columns

Index(['_id', 'ticker', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [33]:
# industry check
benchmark.groupby(["year","quarter","GICS Sector"]).mean().sort_values("delta",ascending=False)

adjClose  \
year quarter GICS Sector                          
2018 1       Health Care              98.328035   
2016 3       Industrials              24.152820   
2018 1       Industrials              67.439527   
2013 3       Industrials              67.113743   
     2       Information Technology    6.108129   
2019 4       Real Estate             126.692553   
2013 4       Consumer Discretionary   14.556689   
2016 3       Information Technology    1.900000   
2015 1       Real Estate             126.120917   
     3       Health Care              29.076010   
2017 3       Industrials              31.866858   
2014 1       Industrials              11.445075   
2017 4       Industrials              20.535059   
2016 2       Communication Services   30.333644   
2015 1       Consumer Staples         43.897694   
2014 3       Industrials              11.430189   
2020 1       Consumer Staples        111.945831   
2015 2       Health Care              67.926118   
2014 2       Consumer Discretionary   96.090000   
2019 4       Financials               16.330974   
2013 3       Health Care              34.121226   
2016 3       Health Care              25.631817   
2013 3       Materials                34.754954   
2020 3       Consumer Staples        195.076905   
2015 1       Utilities                61.387416   
2014 2       Materials                33.297563   
2016 3       Communication Services   44.809152   
2015 2       Consumer Staples         24.446150   

                                     weekly_price_regression_prediction  \
year quarter GICS Sector                                                  
2018 1       Health Care                                     158.506851   
2016 3       Industrials                                      26.098497   
2018 1       Industrials                                     158.506851   
2013 3       Industrials                                      68.401608   
     2       Information Technology                            8.380377   
2019 4       Real Estate                                     147.584846   
2013 4       Consumer Discretionary                           17.884613   
2016 3       Information Technology                            2.056223   
2015 1       Real Estate                                     129.735843   
     3       Health Care                                      30.385764   
2017 3       Industrials                                      42.161404   
2014 1       Industrials                                      76.782867   
2017 4       Industrials                                      64.905609   
2016 2       Communication Services                           31.463837   
2015 1       Consumer Staples                                 45.736747   
2014 3       Industrials                                      27.735804   
2020 1       Consumer Staples                                200.831833   
2015 2       Health Care                                      73.989920   
2014 2       Consumer Discretionary                          113.171361   
2019 4       Financials                                       19.033053   
2013 3       Health Care                                      35.791662   
2016 3       Health Care                                      28.363822   
2013 3       Materials                                        35.462292   
2020 3       Consumer Staples                                202.050406   
2015 1       Utilities                                        64.381346   
2014 2       Materials                                        38.033611   
2016 3       Communication Services                           46.531209   
2015 2       Consumer Staples                                 26.176332   

                                     score     delta  sell_price    d  \
year quarter GICS Sector                                                
2018 1       Health Care              90.0  0.177217  115.605785  0.0   
2016 3       Industrials              

In [34]:
benchmark.groupby(["year","quarter","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)

,year,quarter,ticker,adjClose,weekly_price_regression_prediction,score,delta,sell_price,d,seat,CIK,Security
0,2018,1,WST,98.328035,158.506851,90.0,0.177217,115.605785,0.0,1.000000,105770.0,West Pharmaceutical Services
1,2016,3,PWR,24.152820,26.098497,90.0,0.175292,28.416051,0.0,2.000000,1050915.0,Quanta Services
2,2018,1,KSU,105.606116,158.506851,90.0,0.156245,122.115290,0.0,2.000000,54480.0,Kansas City Southern
3,2013,3,ODFL,29.544552,30.099996,90.0,0.147080,33.889955,0.0,1.000000,878927.0,Old Dominion Freight Line
4,2015,3,RMD,50.558489,52.483298,90.0,0.110377,56.138970,0.0,2.000000,943819.0,ResMed
5,2014,1,LUV,17.782555,76.782867,90.0,0.099576,19.553276,0.0,2.000000,92380.0,Southwest Airlines
6,2013,4,PHM,16.721231,19.617208,90.0,0.081096,18.077249,0.0,2.000000,822416.0,PulteGroup
7,2015,1,SBAC,108.028787,110.197475,90.0,0.080948,116.773481,0.0,1.000000,1034054.0,SBA Communications
8,2013,2,NLOK,9.985781,11.683460,90.0,0.076522,10.749912,0.0,2.000000,849399.0,NortonLifeLock
9,2013,2,MU,9.380000,11.797117,90.0,0.067164,10.010000,0.0,2.000000,723125.0,Micron Technology
